<a href="https://colab.research.google.com/github/moustafa-7/League-of-legends_Bigdata/blob/main/Bigdata_miniprojects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()
import pyspark
import random

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("LOL").getOrCreate()
sc = spark.sparkContext
d = spark.read.json("drive/MyDrive/small_matches.json", multiLine=True)

In [8]:
no_matches = d.count()

## 1. Champions win - pick - ban rates

In [9]:
def champions(row):
  c = []
  bans = []
  for p in row["participants"]:
    if(p["teamId"] == "Blue"): team_number = 0
    else: team_number = 1
    c.append((p["championId"], row["teams"][team_number]["win"]))
  
  for t in row["teams"]:
    for b in t["bans"]:
      if(b["championId"] != None):
        c.append((b["championId"],"ban"))
      # else:
      #   c.append((b["championid"],"ban"))

  return c

In [10]:
champions = d.rdd.flatMap(champions)

In [11]:
import time
def lookup(champions):
  for c in champions.collect():
    print(c)
    time.sleep(0.1)

In [12]:
by_champion = champions.groupByKey().mapValues(list)

In [13]:
# for c, val in by_champion.collect():
#     print(c,val)
#     time.sleep(0.1)

In [14]:
def compute_champion_rates(val):
    ban_count = val.count("ban")
    win_count = val.count("Win")
    lose_count = val.count("Fail")
    total_played_games = win_count+lose_count
    if(total_played_games):
      return [ban_count/no_matches, win_count/total_played_games, lose_count/total_played_games]
    else:
      return [ban_count/no_matches, "never_played", "never_played"]

In [15]:
win_pick_ban_rates = by_champion.map(lambda x:(x[0], compute_champion_rates(x[1])))

In [ ]:
for c in win_pick_ban_rates.collect():
    print(c)
    time.sleep(0.1)

## 2. Champion Synergies or Duos

In [17]:
import itertools
def champion_duos(row):
  duos = []
  team1 = []
  team2 = []
  is_team1_win = False
  is_team2_win = False
  for p in row["participants"]:
    if(p["teamId"] == "Blue"): 
      team_number = 0
    else: 
      team_number = 1
    
    if(team_number == 0):
      team1.append(p["championId"])
      is_team1_win = p["stats"]["win"]
    else:
      team2.append(p["championId"])
      is_team2_win = p["stats"]["win"]
  
  team1 = sorted(team1)
  team2 = sorted(team2)
  for c in list(itertools.combinations(team1, 2)):
    duos.append((c, 1))

  for c in list(itertools.combinations(team2, 2)):
    duos.append((c, 1))
  
  return duos

In [18]:
duos = d.rdd.flatMap(champion_duos)

In [ ]:
from operator import add
duos_count = duos.reduceByKey(add)

In [ ]:
# for c in duos_count.collect():
#     print(c)
#     time.sleep(0.01)

top_duos = duos_count.top(20, key=lambda x: x[1])
for c in top_duos:
    print(c)
    time.sleep(0.01)

(('Ezreal', 'Yuumi'), 40)
(('Ezreal', 'Sett'), 34)
(('Aphelios', 'Thresh'), 33)
(('Ezreal', 'Graves'), 32)
(('Aphelios', 'Sylas'), 29)
(('Bard', 'Ezreal'), 29)
(('Graves', 'Thresh'), 28)
(('Graves', 'Sylas'), 28)
(('Aphelios', 'Lulu'), 27)
(('Aphelios', 'Lee Sin'), 27)
(('Lee Sin', 'Thresh'), 26)
(('Thresh', 'Varus'), 25)
(('Elise', 'Ezreal'), 25)
(('Graves', 'Syndra'), 24)
(('Ezreal', 'Syndra'), 24)
(('Sylas', 'Thresh'), 24)
(('Jayce', 'Varus'), 24)
(('Ezreal', 'Renekton'), 24)
(('Ezreal', 'Lee Sin'), 23)
(('Lee Sin', 'Sylas'), 23)


## 3. Item Win - pick rates

In [ ]:
def get_items(row):
  c = []
  bans = []
  for p in row["participants"]:
    for i in range(7):
      c.append((p["stats"]["item"+str(i)]["name"], p["stats"]["win"]))
  return c

In [ ]:
items = d.rdd.flatMap(get_items).groupByKey().mapValues(list)

In [ ]:

print(len(items.collect()))

190


In [ ]:
def compute_item_rates(val):
  win_count = val.count(True)
  lose_count = val.count(False)
  total_played_games = win_count+lose_count
  return [win_count/total_played_games, len(val)/no_matches]

In [ ]:
item_rates = items.map(lambda x:(x[0], compute_item_rates(x[1])))

In [ ]:
print("item       win_rate        pick_rate")
for item in item_rates.collect():
    print(item)
    time.sleep(0.01)

## 4. Item synergies (with champion, with class)

In [21]:
import json

classes = open("drive/MyDrive/champion_class.json")
champion_classes = json.load(classes)

In [ ]:
champion_classes

In [27]:
def tags(champion_id):
  for c in champion_classes:
    if champion_id==c["championId"]:
      return c["tags"]

In [30]:
print(*tags("Ahri"))

Mage Assassin


In [80]:
def get_items_champion(row):
  c = []
  bans = []
  for p in row["participants"]:
    for i in range(7):
      # c.append((p["stats"]["item"+str(i)]["name"], p["stats"]["win"], p["championId"], *tags(p["championId"])))
      if(tags(p["championId"])):
        c.append((p["stats"]["item"+str(i)]["name"], (p["stats"]["win"], p["championId"],*tags(p["championId"]))))
  return c

In [81]:
items_champion = d.rdd.flatMap(get_items_champion).groupByKey().mapValues(list)

In [82]:
for item in items_champion.collect():
    print(item)
    time.sleep(0.1)

# print(len(items_champion.collect()))

('Shard of True Ice', [(False, 'Karma', 'Mage', 'Support'), (True, 'Karma', 'Mage', 'Support'), (False, 'Soraka', 'Support', 'Mage'), (True, 'Morgana', 'Mage', 'Support'), (True, 'Karma', 'Mage', 'Support'), (False, 'Zyra', 'Mage', 'Support'), (False, 'Karma', 'Mage', 'Support'), (True, 'Yuumi', 'Support', 'Mage'), (False, 'Gragas', 'Fighter', 'Mage'), (True, 'Janna', 'Support', 'Mage'), (True, 'Bard', 'Support', 'Mage'), (True, 'Swain', 'Mage', 'Fighter'), (True, 'Bard', 'Support', 'Mage'), (True, 'Rakan', 'Support'), (True, 'Bard', 'Support', 'Mage'), (False, 'Janna', 'Support', 'Mage'), (True, 'Soraka', 'Support', 'Mage'), (False, 'Lulu', 'Support', 'Mage'), (False, 'Yuumi', 'Support', 'Mage'), (False, 'Morgana', 'Mage', 'Support'), (False, 'Morgana', 'Mage', 'Support'), (False, 'Yuumi', 'Support', 'Mage'), (False, 'Yuumi', 'Support', 'Mage'), (False, 'Soraka', 'Support', 'Mage'), (False, 'Janna', 'Support', 'Mage'), (False, 'Yuumi', 'Support', 'Mage'), (True, 'Lulu', 'Support', 'Ma

KeyboardInterrupt: ignored

In [89]:
def champion_item_synergies(row):
  champion_dict= {}
  for val in row[1]:
    if(val[1] in champion_dict):
      champion_dict[val[1]] += 1
    else:
      champion_dict[val[1]] = 1
    
    if(val[0] == True):
            champion_dict [val[1]] += 0.5

  c = []
  for k, r in sorted(champion_dict.items(), key=lambda x:x[1], reverse=True):
    c.append((k, r))

  return row[0], c[:10]

In [90]:
champion_item_synergy = items_champion.flatMap(lambda x: champion_item_synergies(x))

In [91]:
for item in champion_item_synergy.collect():
    print(item)
    time.sleep(0.1)

Shard of True Ice
[('Yuumi', 149.0), ('Lulu', 127.0), ('Karma', 123.0), ('Bard', 104.0), ('Janna', 92.5), ('Morgana', 58.0), ('Soraka', 33.0), ('Nami', 27.5), ('Fiddlesticks', 26.5), ('Brand', 21.0)]
Athene's Unholy Grail
[('Yuumi', 224.5), ('Karma', 173.5), ('Janna', 107.0), ('Lulu', 90.5), ('Soraka', 45.5), ('Bard', 39.5), ('Ivern', 28.0), ('Rakan', 27.0), ('Nami', 25.5), ('Nidalee', 19.5)]
Ardent Censer
[('Lulu', 278.0), ('Yuumi', 110.5), ('Karma', 103.5), ('Janna', 68.0), ('Soraka', 27.0), ('Ivern', 26.5), ('Rakan', 26.5), ('Nami', 24.5), ('Morgana', 19.5), ('Sona', 9.5)]
Mercury's Treads
[('Graves', 122.5), ('Sylas', 106.0), ('Irelia', 87.0), ('Renekton', 82.5), ('Sett', 70.5), ('Riven', 69.5), ('Bard', 67.5), ('Jayce', 60.0), ('Ekko', 57.5), ('Jax', 52.5)]
Cloth Armor
[('Thresh', 49.0), ('Lulu', 32.5), ('Blitzcrank', 24.0), ('Sett', 21.5), ('Malphite', 18.5), ('Trundle', 18.5), ('Rammus', 16.5), ('Nautilus', 15.5), ('Morgana', 13.5), ('Aatrox', 13)]
Sapphire Crystal
[('Kassadin',

KeyboardInterrupt: ignored